In [20]:
import gensim
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

In [4]:
toxic_data = pd.read_csv('data/corpus/youtube_text.csv', engine='python')

In [5]:
toxic_data = toxic_data.dropna()

In [6]:
toxic_data = toxic_data.sample(5000)

In [7]:
# Generating array of tokens for every comment
comments = []
STOPWORDS = nltk.corpus.stopwords.words('portuguese')
for i, comment in toxic_data['comentario'].iteritems():
    tokenized_comment = nltk.word_tokenize(comment.lower())
    
    clean_comment = [token for token in tokenized_comment if
                     len(token) > 3 and
                     token not in STOPWORDS and
                     token.isalpha()]
    
    comments.append(clean_comment)

In [8]:
cbow = gensim.models.Word2Vec.load('data/word_embeddings/twitter_cbow_100_5')

In [9]:
# Averaging vectors of tokens into vectors of comments
comments_vectors = []
for comment in comments:
    comment_vector = np.zeros((100))
    n_tokens = 1
    for token in comment:
        n_tokens += 1
        try:
            comment_vector += cbow.wv.get_vector(token)
        except KeyError:
            comment_vector += np.zeros((100))
            
    comments_vectors.append(comment_vector)

In [17]:
X = comments_vectors
y = toxic_data['toxico']
clf = LogisticRegression(C=10, solver='lbfgs', max_iter=10000)

In [18]:
cross_val_score(clf, X, y, cv=5, verbose=True, scoring='f1')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.5s finished


array([0.89736399, 0.89385475, 0.90101237, 0.89892716, 0.89141558])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [22]:
clf.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
y_pred = clf.predict(X_test)